# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 8 2022 10:17AM,250276,19,ACG-2102491794,ACG North America LLC,Released
1,Nov 8 2022 10:06AM,250275,10,MSP215607,"Methapharm, Inc.",Released
2,Nov 8 2022 10:06AM,250275,10,MSP215611,"Methapharm, Inc.",Released
3,Nov 8 2022 10:06AM,250275,10,MSP215624,Methapharm-G,Released
4,Nov 8 2022 10:06AM,250275,10,MSP215612,"Methapharm, Inc.",Released
5,Nov 8 2022 10:05AM,250274,10,MSP215583,"Methapharm, Inc.",Released
6,Nov 8 2022 10:05AM,250274,10,MSP215604,"Methapharm, Inc.",Released
7,Nov 8 2022 10:05AM,250274,10,MSP215605,"Methapharm, Inc.",Released
8,Nov 8 2022 10:05AM,250274,10,MSP215606,"Methapharm, Inc.",Released
9,Nov 8 2022 10:05AM,250274,10,MSP215608,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,250272,Released,1
25,250273,Released,10
26,250274,Released,15
27,250275,Released,4
28,250276,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
250272,NaN,NaN,NaN,1.0
250273,NaN,NaN,NaN,10.0
250274,NaN,NaN,NaN,15.0
250275,NaN,NaN,NaN,4.0
250276,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
250147,0.0,0.0,1.0,0.0
250148,0.0,0.0,1.0,0.0
250150,0.0,0.0,0.0,1.0
250194,0.0,0.0,0.0,105.0
250204,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
250147,0.0,0,1,0
250148,0.0,0,1,0
250150,0.0,0,0,1
250194,0.0,0,0,105
250204,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,250147,0.0,0,1,0
1,250148,0.0,0,1,0
2,250150,0.0,0,0,1
3,250194,0.0,0,0,105
4,250204,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,250147,0.0,,1,
1,250148,0.0,,1,
2,250150,0.0,,,1
3,250194,0.0,,,105
4,250204,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 8 2022 10:17AM,250276,19,ACG North America LLC
1,Nov 8 2022 10:06AM,250275,10,"Methapharm, Inc."
3,Nov 8 2022 10:06AM,250275,10,Methapharm-G
5,Nov 8 2022 10:05AM,250274,10,"Methapharm, Inc."
20,Nov 8 2022 9:59AM,250273,10,Eywa Pharma Inc.
30,Nov 8 2022 9:53AM,250272,10,Emerginnova
31,Nov 8 2022 9:49AM,250270,10,"Snap Medical Industries, LLC"
35,Nov 8 2022 9:42AM,250269,10,ISDIN Corporation
37,Nov 8 2022 9:28AM,250267,19,Methapharm-G
38,Nov 8 2022 9:09AM,250264,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Nov 8 2022 10:17AM,250276,19,ACG North America LLC,0.0,,,1
1,Nov 8 2022 10:06AM,250275,10,"Methapharm, Inc.",0.0,,,4
2,Nov 8 2022 10:06AM,250275,10,Methapharm-G,0.0,,,4
3,Nov 8 2022 10:05AM,250274,10,"Methapharm, Inc.",0.0,,,15
4,Nov 8 2022 9:59AM,250273,10,Eywa Pharma Inc.,0.0,,,10
5,Nov 8 2022 9:53AM,250272,10,Emerginnova,0.0,,,1
6,Nov 8 2022 9:49AM,250270,10,"Snap Medical Industries, LLC",0.0,,,4
7,Nov 8 2022 9:42AM,250269,10,ISDIN Corporation,0.0,,,2
8,Nov 8 2022 9:28AM,250267,19,Methapharm-G,0.0,,,1
9,Nov 8 2022 9:09AM,250264,10,ISDIN Corporation,0.0,,,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 10:17AM,250276,19,ACG North America LLC,1,,
1,Nov 8 2022 10:06AM,250275,10,"Methapharm, Inc.",4,,
2,Nov 8 2022 10:06AM,250275,10,Methapharm-G,4,,
3,Nov 8 2022 10:05AM,250274,10,"Methapharm, Inc.",15,,
4,Nov 8 2022 9:59AM,250273,10,Eywa Pharma Inc.,10,,
5,Nov 8 2022 9:53AM,250272,10,Emerginnova,1,,
6,Nov 8 2022 9:49AM,250270,10,"Snap Medical Industries, LLC",4,,
7,Nov 8 2022 9:42AM,250269,10,ISDIN Corporation,2,,
8,Nov 8 2022 9:28AM,250267,19,Methapharm-G,1,,
9,Nov 8 2022 9:09AM,250264,10,ISDIN Corporation,9,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 10:17AM,250276,19,ACG North America LLC,1,,
1,Nov 8 2022 10:06AM,250275,10,"Methapharm, Inc.",4,,
2,Nov 8 2022 10:06AM,250275,10,Methapharm-G,4,,
3,Nov 8 2022 10:05AM,250274,10,"Methapharm, Inc.",15,,
4,Nov 8 2022 9:59AM,250273,10,Eywa Pharma Inc.,10,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 10:17AM,250276,19,ACG North America LLC,1.0,NaN,NaN
1,Nov 8 2022 10:06AM,250275,10,"Methapharm, Inc.",4.0,NaN,NaN
2,Nov 8 2022 10:06AM,250275,10,Methapharm-G,4.0,NaN,NaN
3,Nov 8 2022 10:05AM,250274,10,"Methapharm, Inc.",15.0,NaN,NaN
4,Nov 8 2022 9:59AM,250273,10,Eywa Pharma Inc.,10.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 10:17AM,250276,19,ACG North America LLC,1.0,0.0,0.0
1,Nov 8 2022 10:06AM,250275,10,"Methapharm, Inc.",4.0,0.0,0.0
2,Nov 8 2022 10:06AM,250275,10,Methapharm-G,4.0,0.0,0.0
3,Nov 8 2022 10:05AM,250274,10,"Methapharm, Inc.",15.0,0.0,0.0
4,Nov 8 2022 9:59AM,250273,10,Eywa Pharma Inc.,10.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3753963,126.0,9.0,4.0
12,250262,1.0,0.0,0.0
15,250194,105.0,0.0,0.0
16,250244,0.0,1.0,0.0
19,500543,2.0,0.0,0.0
20,250253,1.0,21.0,0.0
21,1751342,5.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3753963,126.0,9.0,4.0
1,12,250262,1.0,0.0,0.0
2,15,250194,105.0,0.0,0.0
3,16,250244,0.0,1.0,0.0
4,19,500543,2.0,0.0,0.0
5,20,250253,1.0,21.0,0.0
6,21,1751342,5.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,126.0,9.0,4.0
1,12,1.0,0.0,0.0
2,15,105.0,0.0,0.0
3,16,0.0,1.0,0.0
4,19,2.0,0.0,0.0
5,20,1.0,21.0,0.0
6,21,5.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,126.0
1,12,Released,1.0
2,15,Released,105.0
3,16,Released,0.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,0.0,0.0,1.0,0.0,21.0,2.0
Released,126.0,1.0,105.0,0.0,2.0,1.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,0.0,1.0,0.0,21.0,2.0
2,Released,126.0,1.0,105.0,0.0,2.0,1.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,0.0,1.0,0.0,21.0,2.0
2,Released,126.0,1.0,105.0,0.0,2.0,1.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()